# Import des Librairies utiles au projet

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from numpy.random import RandomState
import scipy.stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# A partir des différents datasets à notre disposition sur les courses de Formule 1, comment peut-on prédire quelle écurie gagnera la course ?

### Le dataset que nous avons utilisé est disponible à l'adresse suivante : https://www.kaggle.com/datasets/melissamonfared/formula-1/data
Les données que nous possédons proviennent de multiples jeux de données aux facteurs variés. Chacun d'entre eux contient des informations qu peuvent nous être utiles pour prédire le résultat de la course.

Notre jeu de données contient les csv suivants :
 1. Circuits
 2. Constructor results
 3. Constructor standings
 4. Constructors
 5. drivers
 6. driver standings
 7. Lap times
 8. Pit stops
 9. Qualifying
 10. Races
 11. Results
 12. Seasons
 13. Sprint results
 14. Status

Avant de passer à l'entraînement de notre model, il nous faut déjà visualiser une première fois nos données, pour identifier les variables qui ont le plus d'impacts, si certaines colonnes sont vides, si certaines données sont inutilisables, etc... Cette étape nous aidera à faire les démarches nécessaires au nettoyage de notre dataset afin qu'il soit le plus performant possible.

In [9]:
# Lecture rapide des csv

circuits_df = pd.read_csv('circuits.csv')
print(circuits_df.head())

constructor_results_df = pd.read_csv('constructor_results.csv')
print(constructor_results_df.head())

constructor_standings_df = pd.read_csv('constructor_standings.csv')
print(constructor_standings_df.head())

constructors_df = pd.read_csv('constructors.csv')
print(constructors_df.head())

driver_standings_df = pd.read_csv('driver_standings.csv')
print(driver_standings_df.head())

drivers_df = pd.read_csv('drivers.csv')
print(drivers_df.head())

lap_times_df = pd.read_csv('lap_times.csv')
print(lap_times_df.head())

pit_stops_df = pd.read_csv('pit_stops.csv')
print(pit_stops_df.head())

qualifying_df = pd.read_csv('qualifying.csv')
print(qualifying_df.head())

races_df = pd.read_csv('races.csv')
print(races_df.head())

results_df = pd.read_csv('results.csv')
print(results_df.head())

seasons_df = pd.read_csv('seasons.csv')
print(seasons_df.head())

sprint_results_df = pd.read_csv('sprint_results.csv')
print(sprint_results_df.head())

status_df = pd.read_csv('status.csv')
print(status_df.head())

   circuitId   circuitRef                            name      location  \
0          1  albert_park  Albert Park Grand Prix Circuit     Melbourne   
1          2       sepang    Sepang International Circuit  Kuala Lumpur   
2          3      bahrain   Bahrain International Circuit        Sakhir   
3          4    catalunya  Circuit de Barcelona-Catalunya      Montmeló   
4          5     istanbul                   Istanbul Park      Istanbul   

     country       lat        lng  alt  \
0  Australia -37.84970  144.96800   10   
1   Malaysia   2.76083  101.73800   18   
2    Bahrain  26.03250   50.51060    7   
3      Spain  41.57000    2.26111  109   
4     Turkey  40.95170   29.40500  130   

                                                 url  
0  http://en.wikipedia.org/wiki/Melbourne_Grand_P...  
1  http://en.wikipedia.org/wiki/Sepang_Internatio...  
2  http://en.wikipedia.org/wiki/Bahrain_Internati...  
3  http://en.wikipedia.org/wiki/Circuit_de_Barcel...  
4         http://en.w

FileNotFoundError: [Errno 2] No such file or directory: 'qualifying.csv'

Chargement du merged

In [30]:
dataframes = {}
file_names = [
    "circuits.csv", "constructor_results.csv", "constructor_standings.csv",
    "constructors.csv", "driver_standings.csv", "drivers.csv",
    "lap_times.csv", "pit_stops.csv", "races.csv", "results.csv", "status.csv"
]
na_val = '\\N'

for f_name in file_names:
    try:
        key = f_name.split('.')[0]
        dataframes[key] = pd.read_csv(f_name, na_values=[na_val])
        print(f"Fichier '{f_name}' chargé.")
    except FileNotFoundError:
        # En cas de mauvaise écriture du csv ou non présence des fichiers dans le bon dossier
        print(f"ERREUR INATTENDUE : Fichier non trouvé : '{f_name}'")
        sys.exit()

print("\n--- Début de la fusion ---")

#Extraire les datasets
results = dataframes.get('results')
races = dataframes.get('races')
circuits = dataframes.get('circuits')
drivers = dataframes.get('drivers')
constructors = dataframes.get('constructors')
driver_standings = dataframes.get('driver_standings')
constructor_standings = dataframes.get('constructor_standings')
constructor_results = dataframes.get('constructor_results')
status = dataframes.get('status')

# Renommer les colonnes 'url' en conflit
races = races.rename(columns={'url': 'race_url'})
drivers = drivers.rename(columns={'url': 'driver_url'})
constructors = constructors.rename(columns={'url': 'constructor_url'})
circuits = circuits.rename(columns={'url': 'circuit_url'})

# Fusion dans un ordre logique
df_merged = (
    results
    .merge(races, on="raceId", how="left")
    .merge(circuits, on="circuitId", how="left")
    .merge(drivers, on="driverId", how="left")
    .merge(constructors, on="constructorId", how="left")
    .merge(status, on="statusId", how="left")
    .merge(driver_standings, on=["raceId", "driverId"], how="left", suffixes=('_driver', '_driver_stand'))
    .merge(constructor_standings, on=["raceId", "constructorId"], how="left", suffixes=('_constructor', '_constructor_stand'))
)

# Ajout de constructor_results
if 'constructor_results' in dataframes:
    cr = dataframes.get('constructor_results').copy()
    join_keys = ['raceId', 'constructorId']
    cols_to_prefix = [c for c in cr.columns if c not in join_keys]
    cr = cr.rename(columns={c: f"cr_{c}" for c in cols_to_prefix})
    df_merged = df_merged.merge(cr, on=join_keys, how='left')

# Vérification du résultat
print("\n Fusion complétée avec succès")
print(f" Forme finale du dataset: {df_merged.shape}")




Fichier 'circuits.csv' chargé.
Fichier 'constructor_results.csv' chargé.
Fichier 'constructor_standings.csv' chargé.
Fichier 'constructors.csv' chargé.
Fichier 'driver_standings.csv' chargé.
Fichier 'drivers.csv' chargé.
Fichier 'lap_times.csv' chargé.
Fichier 'pit_stops.csv' chargé.
Fichier 'races.csv' chargé.
Fichier 'results.csv' chargé.
Fichier 'status.csv' chargé.

--- Début de la fusion ---

 Fusion complétée avec succès
 Forme finale du dataset: (26499, 69)


Il y a beaucoup d'écuries différentes alors on a decidé d'affilier les écuries qui ne courent plus actuellement à celles qu'ils les ont rachetées. Les écuries qui n'ont pas été racheté et ne courent plus sont supprimées.

In [33]:
print("Voici tous les constructeurs differents depuis 1950\n")
print(sorted(df_merged['constructorRef'].unique()))


Voici tous les constructeurs differents depuis 1950

['adams', 'afm', 'ags', 'alfa', 'alphatauri', 'alpine', 'alta', 'amon', 'apollon', 'arrows', 'arzani-volpini', 'aston_martin', 'ats', 'bar', 'behra-porsche', 'bellasi', 'benetton', 'bmw', 'bmw_sauber', 'boro', 'brabham', 'brabham-alfa_romeo', 'brabham-brm', 'brabham-climax', 'brabham-ford', 'brabham-repco', 'brawn', 'brm', 'brm-ford', 'bromme', 'brp', 'bugatti', 'butterworth', 'caterham', 'cisitalia', 'coloni', 'connaught', 'connew', 'cooper', 'cooper-alfa_romeo', 'cooper-ats', 'cooper-borgward', 'cooper-brm', 'cooper-castellotti', 'cooper-climax', 'cooper-ferrari', 'cooper-ford', 'cooper-maserati', 'cooper-osca', 'dallara', 'de_tomaso-alfa_romeo', 'de_tomaso-ferrari', 'de_tomaso-osca', 'deidt', 'del_roy', 'derrington', 'dunn', 'eagle-climax', 'eagle-weslake', 'elder', 'emeryson', 'emw', 'enb', 'ensign', 'epperly', 'era', 'eurobrun', 'ewing', 'ferguson', 'ferrari', 'fittipaldi', 'fondmetal', 'footwork', 'force_india', 'forti', 'fraze

On affilie les anciennes écuries aux nouvelles si elles existent toujours

In [39]:
constructor_mapping = {
    # Anciennes -> Actuelles / Groupes
    'adams': 'unknown',
    'afm': 'unknown',
    'ags': 'unknown',
    'alfa': 'alfa_romeo',
    'alphatauri': 'alphatauri',
    'alpine': 'alpine',
    'alta': 'unknown',
    'amon': 'unknown',
    'apollon': 'unknown',
    'arrows': 'arrows',
    'arzani-volpini': 'unknown',
    'aston_martin': 'aston_martin',
    'ats': 'ats',
    'bar': 'arrows',
    'behra-porsche': 'unknown',
    'bellasi': 'unknown',
    'benetton': 'alpine',
    'bmw': 'bmw',
    'bmw_sauber': 'sauber',
    'boro': 'unknown',
    'brabham': 'brabham',
    'brabham-alfa_romeo': 'brabham',
    'brabham-brm': 'brabham',
    'brabham-climax': 'brabham',
    'brabham-ford': 'brabham',
    'brabham-repco': 'brabham',
    'brawn': 'mercedes',
    'brm': 'brm',
    'brm-ford': 'brm',
    'bromme': 'unknown',
    'brp': 'unknown',
    'bugatti': 'unknown',
    'butterworth': 'unknown',
    'caterham': 'caterham',
    'cisitalia': 'unknown',
    'coloni': 'unknown',
    'connaught': 'unknown',
    'connew': 'unknown',
    'cooper': 'cooper',
    'cooper-alfa_romeo': 'cooper',
    'cooper-ats': 'cooper',
    'cooper-borgward': 'cooper',
    'cooper-brm': 'cooper',
    'cooper-castellotti': 'cooper',
    'cooper-climax': 'cooper',
    'cooper-ferrari': 'cooper',
    'cooper-ford': 'cooper',
    'cooper-maserati': 'cooper',
    'cooper-osca': 'cooper',
    'dallara': 'dallara',
    'de_tomaso-alfa_romeo': 'de_tomaso',
    'de_tomaso-ferrari': 'de_tomaso',
    'de_tomaso-osca': 'de_tomaso',
    'deidt': 'unknown',
    'del_roy': 'unknown',
    'derrington': 'unknown',
    'dunn': 'unknown',
    'eagle-climax': 'eagle',
    'eagle-weslake': 'eagle',
    'elder': 'unknown',
    'emeryson': 'unknown',
    'emw': 'unknown',
    'enb': 'unknown',
    'ensign': 'ensign',
    'epperly': 'unknown',
    'era': 'era',
    'eurobrun': 'unknown',
    'ewing': 'unknown',
    'ferguson': 'unknown',
    'ferrari': 'ferrari',
    'fittipaldi': 'unknown',
    'fondmetal': 'unknown',
    'footwork': 'arrows',
    'force_india': 'force_india',
    'forti': 'unknown',
    'frazer_nash': 'unknown',
    'fry': 'unknown',
    'gilby': 'unknown',
    'gordini': 'gordini',
    'haas': 'haas',
    'hall': 'unknown',
    'hesketh': 'hesketh',
    'hill': 'unknown',
    'honda': 'honda',
    'hrt': 'unknown',
    'hwm': 'hwm',
    'iso_marlboro': 'iso_marlboro',
    'jaguar': 'red_bull',
    'jbw': 'unknown',
    'jordan': 'aston_martin',
    'kauhsen': 'unknown',
    'klenk': 'unknown',
    'kojima': 'unknown',
    'kurtis_kraft': 'unknown',
    'kuzma': 'unknown',
    'lago': 'unknown',
    'lambo': 'unknown',
    'lancia': 'unknown',
    'langley': 'unknown',
    'larrousse': 'unknown',
    'lds': 'lds',
    'lds-alfa_romeo': 'lds',
    'lds-climax': 'lds',
    'lec': 'unknown',
    'lesovsky': 'unknown',
    'leyton': 'aston_martin',
    'life': 'unknown',
    'ligier': 'ligier',
    'lola': 'lola',
    'lotus-borgward': 'lotus',
    'lotus-brm': 'lotus',
    'lotus-climax': 'lotus',
    'lotus-ford': 'lotus',
    'lotus-maserati': 'lotus',
    'lotus-pw': 'lotus',
    'lotus_f1': 'alpine',
    'lotus_racing': 'alpine',
    'lyncar': 'unknown',
    'maki': 'unknown',
    'manor': 'manor',
    'march': 'march',
    'march-alfa_romeo': 'march',
    'march-ford': 'march',
    'marchese': 'unknown',
    'martini': 'martini',
    'marussia': 'manor',
    'maserati': 'maserati',
    'matra': 'matra',
    'matra-ford': 'matra',
    'mbm': 'unknown',
    'mcguire': 'unknown',
    'mclaren': 'mclaren',
    'mclaren-alfa_romeo': 'mclaren',
    'mclaren-brm': 'mclaren',
    'mclaren-ford': 'mclaren',
    'mclaren-seren': 'mclaren',
    'mercedes': 'mercedes',
    'merzario': 'unknown',
    'meskowski': 'unknown',
    'mf1': 'unknown',
    'milano': 'unknown',
    'minardi': 'minardi',
    'moda': 'unknown',
    'moore': 'unknown',
    'nichels': 'unknown',
    'olson': 'unknown',
    'onyx': 'onyx',
    'osca': 'osca',
    'osella': 'osella',
    'pacific': 'pacific',
    'pankratz': 'unknown',
    'parnelli': 'unknown',
    'pawl': 'unknown',
    'penske': 'penske',
    'phillips': 'unknown',
    'politoys': 'unknown',
    'porsche': 'porsche',
    'prost': 'prost',
    'protos': 'unknown',
    'racing_point': 'aston_martin',
    'rae': 'unknown',
    'ram': 'ram',
    'rb': 'red_bull',
    're': 're',
    'rebaque': 'unknown',
    'red_bull': 'red_bull',
    'renault': 'alpine',
    'rial': 'rial',
    'sauber': 'sauber',
    'scarab': 'unknown',
    'schroeder': 'unknown',
    'scirocco': 'unknown',
    'shadow': 'shadow',
    'shadow-ford': 'shadow',
    'shadow-matra': 'shadow',
    'shannon': 'unknown',
    'sherman': 'unknown',
    'simca': 'unknown',
    'simtek': 'simtek',
    'snowberger': 'unknown',
    'spirit': 'spirit',
    'spyker': 'force_india',
    'spyker_mf1': 'force_india',
    'stebro': 'unknown',
    'stevens': 'unknown',
    'stewart': 'stewart',
    'super_aguri': 'unknown',
    'surtees': 'surtees',
    'sutton': 'unknown',
    'team_lotus': 'alpine',
    'tec-mec': 'unknown',
    'tecno': 'tecno',
    'theodore': 'theodore',
    'token': 'unknown',
    'toleman': 'benetton',
    'tomaso': 'de_tomaso',
    'toro_rosso': 'alphatauri',
    'toyota': 'toyota',
    'trevis': 'unknown',
    'trojan': 'unknown',
    'turner': 'unknown',
    'tyrrell': 'mercedes',
    'vanwall': 'vanwall',
    'veritas': 'unknown',
    'vhristensen': 'unknown',
    'virgin': 'manor',
    'watson': 'unknown',
    'wetteroth': 'unknown',
    'williams': 'williams',
    'wolf': 'arrows',
    'zakspeed': 'zakspeed'
}

# Appliquer le mapping
df_merged['constructor_grouped'] = df_merged['constructorRef'].replace(constructor_mapping)

# Vérifier
print(df_merged[['constructorRef', 'constructor_grouped']].head())


  constructorRef constructor_grouped
0        mclaren             mclaren
1     bmw_sauber              sauber
2       williams            williams
3        renault              alpine
4        mclaren             mclaren


On garde uniquement celle qui participent actuellement

In [42]:
# Mapping complet vers les écuries actuelles
constructor_to_current = {
    # Mercedes et héritiers
    'brawn': 'mercedes',
    'tyrrell': 'mercedes',
    'mercedes': 'mercedes',

    # Red Bull et AlphaTauri
    'red_bull': 'red_bull',
    'toro_rosso': 'alphatauri',
    'alphatauri': 'alphatauri',
    'rb': 'red_bull',
    'jaguar': 'red_bull',

    # Ferrari
    'ferrari': 'ferrari',
    'cooper-ferrari': 'ferrari',

    # Alpine (incl. Renault, Benetton, Lotus)
    'alpine': 'alpine',
    'renault': 'alpine',
    'benetton': 'alpine',
    'lotus_f1': 'alpine',
    'team_lotus': 'alpine',
    'lotus_racing': 'alpine',

    # McLaren
    'mclaren': 'mclaren',
    'mclaren-alfa_romeo': 'mclaren',
    'mclaren-brm': 'mclaren',
    'mclaren-ford': 'mclaren',
    'mclaren-seren': 'mclaren',

    # Aston Martin (incl. Racing Point, Jordan)
    'aston_martin': 'aston_martin',
    'racing_point': 'aston_martin',
    'jordan': 'aston_martin',
    'leyton': 'aston_martin',

    # Alfa Romeo
    'alfa': 'alfa_romeo',
    'sauber': 'alfa_romeo',
    'bmw_sauber': 'alfa_romeo',

    # Haas
    'haas': 'haas',

    # Williams
    'williams': 'williams',

    # Les autres écuries historiques sans successeur actuel
    'unknown': 'unknown'
}

# Remplacer les anciens noms par les écuries actuelles
df_merged['constructor_current'] = df_merged['constructor_grouped'].replace(constructor_to_current)

# Vérification
df_merged['constructor_current'] = df_merged['constructor_grouped'].map(constructor_to_current).fillna('unknown')

print(df_merged[['constructorRef', 'constructor_grouped', 'constructor_current']].head(25))
print(len(df_merged['constructor_current'].unique()))



   constructorRef constructor_grouped constructor_current
0         mclaren             mclaren             mclaren
1      bmw_sauber              sauber          alfa_romeo
2        williams            williams            williams
3         renault              alpine              alpine
4         mclaren             mclaren             mclaren
5        williams            williams            williams
6      toro_rosso          alphatauri          alphatauri
7         ferrari             ferrari             ferrari
8      bmw_sauber              sauber          alfa_romeo
9          toyota              toyota             unknown
10    super_aguri             unknown             unknown
11        renault              alpine              alpine
12        ferrari             ferrari             ferrari
13       red_bull            red_bull            red_bull
14         toyota              toyota             unknown
15    force_india         force_india             unknown
16       red_b

In [46]:
print("Ecuries actuelles\n")
print(df_merged['constructor_current'].unique())

Ecuries actuelles

['mclaren' 'alfa_romeo' 'williams' 'alpine' 'alphatauri' 'ferrari'
 'unknown' 'red_bull' 'aston_martin' 'mercedes' 'haas']


on supprime les écuries qui ne participent pas actuellement aux GP

In [49]:
df_merged = df_merged[df_merged['constructor_current'] != 'unknown']


la on a un dataframe avec uniquement les constructors actuels et leurs affiliations précedentes, les autres lignes sont supprimées